## 1. 환경 설정 및 라이브러리 import

In [38]:
# 즉시 실행
torch.cuda.empty_cache()
torch.cuda.synchronize()

# 다시 확인
print(f"Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

Allocated: 0.14 GB
Cached: 0.23 GB


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, GATConv, GCNConv, SAGEConv, GraphConv
from torch_geometric.utils import scatter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb

import time
import pickle
import io
import warnings
warnings.filterwarnings('ignore')

# 디바이스 설정
device = torch.device('cuda')  # PyTorch Geometric MPS 호환성 문제로 CPU 사용
torch.set_num_threads(8)

print(f"   디바이스: {device}")

   디바이스: cuda


## 2. 데이터 로드

In [40]:
class CPUUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

# 데이터 로드
data_path = '../data/processed_data/processed_data_GNN.pkl'
with open(data_path, 'rb') as f:
    data = pickle.load(f)

# GPU로 이동
data = data.to('cuda')

print(f"   사용자 수: {len(data.user_id_mapping):,}")
print(f"   음식 수: {len(data.food_id_mapping):,}")
print(f"   그래프 엣지 타입: {list(data.edge_index_dict.keys())}")
print(f"   노드 타입: {list(data.x_dict.keys())}")

for node_type, features in data.x_dict.items():
    print(f"   {node_type} 특성: {features.shape}")

if ('user', 'healthness', 'food') in data.edge_index_dict:
    health_edges = data[('user', 'healthness', 'food')].edge_index.shape[1]
    health_scores = data[('user', 'healthness', 'food')].edge_attr
    print(f"   건강 관계: {health_edges:,}개")
    print(f"   건강 점수 범위: [{health_scores.min():.3f}, {health_scores.max():.3f}]")
else:
    print("   ❌ healthness 엣지를 찾을 수 없음")

   사용자 수: 20,820
   음식 수: 31,458
   그래프 엣지 타입: [('user', 'eats', 'food'), ('food', 'contains', 'ingredient'), ('food', 'eaten_at', 'time'), ('user', 'healthness', 'food'), ('food', 'similar', 'food'), ('food', 'rev_eats', 'user'), ('ingredient', 'rev_contains', 'food'), ('time', 'rev_eaten_at', 'food'), ('food', 'rev_healthness', 'user')]
   노드 타입: ['user', 'food', 'ingredient', 'time']
   user 특성: torch.Size([20820, 29])
   food 특성: torch.Size([31458, 17])
   ingredient 특성: torch.Size([3284, 101])
   time 특성: torch.Size([4, 4])
   건강 관계: 262,270개
   건강 점수 범위: [0.295, 0.958]


## 3. 모델 정의

### 3.1 Traditional ML Models

In [41]:
def prepare_ml_data(data, sample_size=50000):
    eats_edge_index = data[('user', 'eats', 'food')].edge_index
    eats_edge_attr = data[('user', 'eats', 'food')].edge_attr

    user_indices = eats_edge_index[0].cpu().numpy()
    food_indices = eats_edge_index[1].cpu().numpy()
    eats_scores = eats_edge_attr.cpu().numpy()

    user_features = data.x_dict['user'].cpu().numpy()
    food_features = data.x_dict['food'].cpu().numpy()

    # 샘플링
    sample_indices = np.random.choice(len(eats_scores), min(sample_size, len(eats_scores)), replace=False)
    threshold = np.median(eats_scores)

    # 특성 결합 및 고급 특성 엔지니어링
    X = []
    y = []

    for idx in sample_indices:
        user_idx = user_indices[idx]
        food_idx = food_indices[idx]
        
        # 기본 특성
        user_feat = user_features[user_idx]
        food_feat = food_features[food_idx]
        
        # 고급 특성 엔지니어링
        min_len = min(len(user_feat), len(food_feat))
        interaction = user_feat[:min_len] * food_feat[:min_len]
        difference = np.abs(user_feat[:min_len] - food_feat[:min_len])
        
        # 통계 특성
        user_stats = [user_feat.mean(), user_feat.std(), user_feat.max(), user_feat.min()]
        food_stats = [food_feat.mean(), food_feat.std(), food_feat.max(), food_feat.min()]
        
        # 모든 특성 결합
        combined = np.concatenate([
            user_feat, food_feat, interaction, difference, user_stats, food_stats
        ])
        
        X.append(combined)
        y.append(1 if eats_scores[idx] > threshold else 0)

    return np.array(X), np.array(y), threshold, sample_indices


### 3.2 Deep Learning Models

In [42]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64]):
        super().__init__()
        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3)
            ])
            prev_dim = hidden_dim

        layers.extend([nn.Linear(prev_dim, 1), nn.Sigmoid()])
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class AdvancedMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[512, 256, 128]):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dims[0])
        self.bn_input = nn.BatchNorm1d(hidden_dims[0])

        self.hidden_layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for i in range(len(hidden_dims) - 1):
            self.hidden_layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dims[i+1]))

        self.output_layer = nn.Linear(hidden_dims[-1], 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.bn_input(self.input_layer(x)))
        x = self.dropout(x)

        for layer, bn in zip(self.hidden_layers, self.batch_norms):
            residual = x
            x = F.relu(bn(layer(x)))
            x = self.dropout(x)
            if x.shape == residual.shape:
                x = x + residual

        return torch.sigmoid(self.output_layer(x))


### 3.3 GNN Baseline Models

In [18]:
class VanillaGNN(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()

        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })

        self.conv1 = HeteroConv({
            ('user', 'eats', 'food'): GraphConv(hidden_channels, out_channels),
            ('food', 'rev_eats', 'user'): GraphConv(hidden_channels, out_channels)
        })

        self.conv2 = HeteroConv({
            ('user', 'eats', 'food'): GraphConv(out_channels, out_channels),
            ('food', 'rev_eats', 'user'): GraphConv(out_channels, out_channels)
        })

        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x_dict, edge_index_dict, edge_label_index, **kwargs):
        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}

        current_x = self.conv1(current_x, edge_index_dict)
        current_x = {k: F.relu(v) for k, v in current_x.items()}

        current_x = self.conv2(current_x, edge_index_dict)
        current_x = {k: F.relu(v) for k, v in current_x.items()}

        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]

        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.decoder(combined).squeeze()

class GraphSAGE(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()

        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })

        self.conv1 = HeteroConv({
            ('user', 'eats', 'food'): SAGEConv(hidden_channels, out_channels),
            ('food', 'rev_eats', 'user'): SAGEConv(hidden_channels, out_channels)
        })

        self.conv2 = HeteroConv({
            ('user', 'eats', 'food'): SAGEConv(out_channels, out_channels),
            ('food', 'rev_eats', 'user'): SAGEConv(out_channels, out_channels)
        })

        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x_dict, edge_index_dict, edge_label_index, **kwargs):
        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}

        current_x = self.conv1(current_x, edge_index_dict)
        current_x = {k: F.relu(v) for k, v in current_x.items()}

        current_x = self.conv2(current_x, edge_index_dict)
        current_x = {k: F.relu(v) for k, v in current_x.items()}

        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]

        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.decoder(combined).squeeze()

class GAT_NoHealth(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()

        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })

        self.conv = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })

        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )

        self.batch_norms = nn.ModuleDict({
            node_type: nn.BatchNorm1d(out_channels) for node_type in metadata[0]
        })

    def forward(self, x_dict, edge_index_dict, edge_label_index, **kwargs):
        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}

        current_x = self.conv(current_x, edge_index_dict)

        for node_type in current_x.keys():
            if node_type in self.batch_norms:
                current_x[node_type] = F.relu(self.batch_norms[node_type](current_x[node_type]))

        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]

        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.decoder(combined).squeeze()


### 3.4 Ablation Study Models

In [19]:
class HealthAware_NoAttention(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()

        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })

        self.conv = HeteroConv({
            ('user', 'eats', 'food'): GraphConv(hidden_channels, out_channels),
            ('food', 'rev_eats', 'user'): GraphConv(hidden_channels, out_channels)
        })

        # 간단한 건강 선호도 네트워크 (어텐션 없음)
        self.health_preference_net = nn.Sequential(
            nn.Linear(out_channels, 1),
            nn.Sigmoid()
        )

        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x_dict, edge_index_dict, edge_label_index,
                health_edge_index=None, health_scores=None, training=True):

        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}

        current_x = self.conv(current_x, edge_index_dict)
        current_x = {k: F.relu(v) for k, v in current_x.items()}

        # 간단한 건강 정보 통합 (어텐션 없음)
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            user_health_prefs = self.health_preference_net(user_embeddings)

            avg_health = health_scores.mean()
            health_scale = 0.1 * avg_health
            current_x['food'] = current_x['food'] + health_scale

        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]

        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.decoder(combined).squeeze()

class HealthAware_NoLoss(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()

        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })

        self.conv = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })

        self.health_preference_net = nn.Sequential(
            nn.Linear(out_channels, out_channels // 2),
            nn.ReLU(),
            nn.Linear(out_channels // 2, 1),
            nn.Sigmoid()
        )

        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )

        self.batch_norms = nn.ModuleDict({
            node_type: nn.BatchNorm1d(out_channels) for node_type in metadata[0]
        })

    def forward(self, x_dict, edge_index_dict, edge_label_index,
                health_edge_index=None, health_scores=None, training=True):

        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}

        current_x = self.conv(current_x, edge_index_dict)

        for node_type in current_x.keys():
            if node_type in self.batch_norms:
                current_x[node_type] = F.relu(self.batch_norms[node_type](current_x[node_type]))

        # 건강 어텐션 (손실 없이)
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            food_embeddings = current_x['food']

            user_health_prefs = self.health_preference_net(user_embeddings)

            user_indices_health = health_edge_index[0]
            food_indices_health = health_edge_index[1]

            personal_weights = user_health_prefs[user_indices_health].squeeze()
            enhanced_health = torch.sigmoid(health_scores * 2.0)
            final_weights = personal_weights * enhanced_health

            health_updates = scatter(
                final_weights, food_indices_health, dim=0,
                dim_size=food_embeddings.size(0), reduce='mean'
            )

            health_scale = 0.2
            health_expanded = health_updates.unsqueeze(1).expand(-1, current_x['food'].size(1))
            current_x['food'] = current_x['food'] + health_scale * health_expanded

        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]

        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.decoder(combined).squeeze()


### 3.5 SOTA Models

In [20]:
class TransformerEnhancedGNN_Fixed(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims, num_heads=8):
        super().__init__()
        
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.num_heads = num_heads
        
        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })
        
        self.conv = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })
        
        self.multihead_attn = nn.MultiheadAttention(
            embed_dim=out_channels,
            num_heads=4,
            dropout=0.1,
            batch_first=True
        )
        
        self.pos_encoding = nn.Parameter(torch.randn(1000, out_channels))
        self.layer_norm = nn.LayerNorm(out_channels)
        
        self.health_preference_net = nn.Sequential(
            nn.Linear(out_channels, out_channels // 2),
            nn.ReLU(),
            nn.Linear(out_channels // 2, 1),
            nn.Sigmoid()
        )
        
        self.ffn = nn.Sequential(
            nn.Linear(out_channels, out_channels * 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(out_channels * 2, out_channels),
            nn.Dropout(0.1)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x_dict, edge_index_dict, edge_label_index, health_edge_index=None, health_scores=None, training=True):
        
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.input_projections[node_type](x)
        
        x_dict = self.conv(x_dict, edge_index_dict)
        
        user_embeddings = x_dict['user']
        food_embeddings = x_dict['food']
        
        x_combined = torch.cat([user_embeddings, food_embeddings], dim=0)
        
        seq_len = x_combined.size(0)
        if seq_len <= self.pos_encoding.size(0):
            pos_enc = self.pos_encoding[:seq_len]
            x_combined = x_combined + pos_enc
        
        x_norm = self.layer_norm(x_combined)
        
        x_expanded = x_norm.unsqueeze(0)
        attn_out, _ = self.multihead_attn(x_expanded, x_expanded, x_expanded)
        x_attn = attn_out.squeeze(0)
        
        x_residual = x_norm + x_attn
        
        x_ffn = self.ffn(x_residual)
        x_output = x_residual + x_ffn
        
        n_users = user_embeddings.size(0)
        x_dict['user'] = x_output[:n_users]
        x_dict['food'] = x_output[n_users:]
        
        user_indices, food_indices = edge_label_index
        user_emb = x_dict['user'][user_indices]
        food_emb = x_dict['food'][food_indices]
        
        edge_emb = torch.cat([user_emb, food_emb], dim=1)
        predictions = self.decoder(edge_emb).squeeze()
        
        return predictions

class ContrastiveLearningGNN(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims, temperature=0.07):
        super().__init__()
        
        self.temperature = temperature
        
        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })
        
        # 기본 인코더
        self.encoder = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })
        
        # Contrastive projection heads
        self.user_projector = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 64)
        )
        
        self.food_projector = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 64)
        )
        
        # Health-aware projector
        self.health_projector = nn.Sequential(
            nn.Linear(out_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Linear(hidden_channels // 2, 32)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )
        
        self.batch_norms = nn.ModuleDict({
            node_type: nn.BatchNorm1d(out_channels) for node_type in metadata[0]
        })

    def contrastive_loss(self, user_proj, food_proj, positive_pairs):
        """InfoNCE 대조 손실 계산"""
        # L2 normalize
        user_proj = F.normalize(user_proj, dim=-1)
        food_proj = F.normalize(food_proj, dim=-1)
        
        # Compute similarity matrix
        similarity = torch.matmul(user_proj, food_proj.T) / self.temperature
        
        # Create positive mask
        batch_size = user_proj.size(0)
        positive_mask = torch.zeros(batch_size, batch_size, device=user_proj.device)
        for i, j in positive_pairs:
            if i < batch_size and j < batch_size:
                positive_mask[i, j] = 1
        
        # InfoNCE loss
        exp_sim = torch.exp(similarity)
        pos_exp_sim = exp_sim * positive_mask
        neg_exp_sim = exp_sim * (1 - positive_mask)
        
        pos_sum = pos_exp_sim.sum(dim=1, keepdim=True).clamp(min=1e-8)
        neg_sum = neg_exp_sim.sum(dim=1, keepdim=True)
        
        loss = -torch.log(pos_sum / (pos_sum + neg_sum)).mean()
        return loss

    def forward(self, x_dict, edge_index_dict, edge_label_index,
                health_edge_index=None, health_scores=None, training=True):
        
        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}
        
        current_x = self.encoder(current_x, edge_index_dict)
        
        for node_type in current_x.keys():
            if node_type in self.batch_norms:
                current_x[node_type] = F.relu(self.batch_norms[node_type](current_x[node_type]))
        
        # 건강 정보 통합
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            food_embeddings = current_x['food']
            
            # Health-aware enhancement
            user_health_proj = self.health_projector(user_embeddings)
            food_health_proj = self.health_projector(food_embeddings)
            
            user_indices_health = health_edge_index[0]
            food_indices_health = health_edge_index[1]
            
            health_enhanced = torch.sigmoid(health_scores * 2.0)
            health_updates = scatter(
                health_enhanced, food_indices_health, dim=0,
                dim_size=food_embeddings.size(0), reduce='mean'
            )
            
            health_scale = 0.25
            health_expanded = health_updates.unsqueeze(1).expand(-1, current_x['food'].size(1))
            current_x['food'] = current_x['food'] + health_scale * health_expanded
        
        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]
        
        # Store projections for contrastive learning
        if training:
            self.user_projections = self.user_projector(user_emb)
            self.food_projections = self.food_projector(food_emb)
        
        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.decoder(combined).squeeze()

class MetaLearningGNN(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()
        
        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })
        
        # Meta-learnable base network
        self.meta_conv = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })
        
        # Personal adaptation networks
        self.user_adaptation = nn.Sequential(
            nn.Linear(out_channels, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )
        
        self.food_adaptation = nn.Sequential(
            nn.Linear(out_channels, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )
        
        # Health personalization network
        self.health_personalization = nn.Sequential(
            nn.Linear(out_channels + 1, out_channels),  # +1 for health score
            nn.ReLU(),
            nn.Linear(out_channels, out_channels),
            nn.Sigmoid()
        )
        
        # Dynamic decoder (personalized)
        self.meta_decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )
        
        # Personal preference learning
        self.preference_embedding = nn.Embedding(10000, 32)  # User preference embeddings
        self.preference_fusion = nn.Linear(out_channels + 32, out_channels)

    def adapt_to_user(self, user_embeddings, user_history=None):
        """사용자별 적응형 임베딩 생성"""
        adapted_user = self.user_adaptation(user_embeddings)
        
        # Add personal preference if available
        if user_history is not None:
            pref_emb = self.preference_embedding(user_history)
            combined = torch.cat([adapted_user, pref_emb], dim=-1)
            adapted_user = self.preference_fusion(combined)
        
        return adapted_user

    def forward(self, x_dict, edge_index_dict, edge_label_index,
                health_edge_index=None, health_scores=None, training=True, user_history=None):
        
        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}
        
        current_x = self.meta_conv(current_x, edge_index_dict)
        current_x = {k: F.relu(v) for k, v in current_x.items()}
        
        # Personal adaptation
        current_x['user'] = self.adapt_to_user(current_x['user'], user_history)
        current_x['food'] = self.food_adaptation(current_x['food'])
        
        # Health-aware personalization
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            food_embeddings = current_x['food']
            
            user_indices_health = health_edge_index[0]
            food_indices_health = health_edge_index[1]
            
            # Personal health preferences
            health_context = torch.cat([
                food_embeddings[food_indices_health],
                health_scores.unsqueeze(1)
            ], dim=1)
            
            personal_health_weights = self.health_personalization(health_context)
            
            health_updates = scatter(
                personal_health_weights.squeeze(), food_indices_health, dim=0,
                dim_size=food_embeddings.size(0), reduce='mean'
            )
            
            health_scale = 0.2
            current_x['food'] = current_x['food'] * (1 + health_scale * health_updates.unsqueeze(1))
        
        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]
        
        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.meta_decoder(combined).squeeze()


#### 3.5.5. Multi-Task Learning Framework

In [21]:
class MultiTaskHealthGNN(nn.Module):
    """멀티태스크 학습 프레임워크 (추천 + 건강예측 + 영양분석)"""
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()
        
        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })
        
        # Shared feature extractor
        self.shared_encoder = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })
        
        # Task-specific heads
        # Task 1: Preference Prediction (Main task)
        self.preference_head = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )
        
        # Task 2: Health Score Prediction
        self.health_prediction_head = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1)  # Regression for health score
        )
        
        # Task 3: Nutritional Category Classification
        self.nutrition_classification_head = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 5),  # 5 nutritional categories
            nn.Softmax(dim=-1)
        )
        
        # Task 4: User Health Risk Assessment
        self.health_risk_head = nn.Sequential(
            nn.Linear(out_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels // 2, 3),  # Low, Medium, High risk
            nn.Softmax(dim=-1)
        )
        
        # Health-aware attention with multi-task context
        self.health_attention = nn.MultiheadAttention(
            embed_dim=out_channels,
            num_heads=4,
            dropout=0.1,
            batch_first=True
        )
        
        # Task balancing weights (learnable)
        self.task_weights = nn.Parameter(torch.ones(4))
        
        self.batch_norms = nn.ModuleDict({
            node_type: nn.BatchNorm1d(out_channels) for node_type in metadata[0]
        })

    def forward(self, x_dict, edge_index_dict, edge_label_index,
                health_edge_index=None, health_scores=None, training=True, return_all_tasks=False):
        
        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}
        
        # Shared encoding
        current_x = self.shared_encoder(current_x, edge_index_dict)
        
        for node_type in current_x.keys():
            if node_type in self.batch_norms:
                current_x[node_type] = F.relu(self.batch_norms[node_type](current_x[node_type]))
        
        # Health-aware processing
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            food_embeddings = current_x['food']
            
            # Multi-task health attention
            user_exp = user_embeddings.unsqueeze(0)
            food_exp = food_embeddings.unsqueeze(0)
            
            health_attn_out, _ = self.health_attention(user_exp, food_exp, food_exp)
            enhanced_user = health_attn_out.squeeze(0)
            
            # Health integration
            user_indices_health = health_edge_index[0]
            food_indices_health = health_edge_index[1]
            
            health_weights = torch.sigmoid(health_scores * 2.0)
            health_updates = scatter(
                health_weights, food_indices_health, dim=0,
                dim_size=food_embeddings.size(0), reduce='mean'
            )
            
            health_scale = 0.25
            health_expanded = health_updates.unsqueeze(1).expand(-1, current_x['food'].size(1))
            current_x['food'] = current_x['food'] + health_scale * health_expanded
            current_x['user'] = enhanced_user
        
        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]
        
        # Task 1: Preference Prediction (Main)
        combined = torch.cat([user_emb, food_emb], dim=-1)
        preference_pred = self.preference_head(combined).squeeze()
        
        if not return_all_tasks:
            return preference_pred
        
        # All tasks for multi-task training
        tasks_output = {}
        
        # Task 1: Preference (already computed)
        tasks_output['preference'] = preference_pred
        
        # Task 2: Health Score Prediction
        tasks_output['health_score'] = self.health_prediction_head(combined).squeeze()
        
        # Task 3: Nutritional Classification (food-based)
        tasks_output['nutrition_category'] = self.nutrition_classification_head(food_emb)
        
        # Task 4: User Health Risk (user-based)
        tasks_output['health_risk'] = self.health_risk_head(user_emb)
        
        return tasks_output


#### 3.5.6. Self-Supervised Pre-training GNN

In [22]:
class SelfSupervisedGNN(nn.Module):
    """자기지도 학습 기반 사전훈련 GNN (GraphMAE/SimGRACE 스타일)"""
    def __init__(self, hidden_channels, out_channels, metadata, input_dims, mask_ratio=0.15):
        super().__init__()
        
        self.mask_ratio = mask_ratio
        
        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })
        
        # Encoder
        self.encoder = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })
        
        # Decoder for reconstruction
        self.decoder = nn.ModuleDict({
            node_type: nn.Sequential(
                nn.Linear(out_channels, hidden_channels),
                nn.ReLU(),
                nn.Linear(hidden_channels, input_dims[node_type])
            ) for node_type in metadata[0]
        })
        
        # Contrastive projection
        self.contrastive_projector = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 128)
        )
        
        # Health-aware components
        self.health_attention = nn.MultiheadAttention(
            embed_dim=out_channels,
            num_heads=4,
            dropout=0.1,
            batch_first=True
        )
        
        # Final task head (for downstream task)
        self.task_head = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )
        
        self.batch_norms = nn.ModuleDict({
            node_type: nn.BatchNorm1d(out_channels) for node_type in metadata[0]
        })

    def mask_features(self, x_dict):
        """특성 마스킹 (자기지도 학습용)"""
        masked_x = {}
        masks = {}
        
        for node_type, features in x_dict.items():
            mask = torch.rand(features.shape, device=features.device) < self.mask_ratio
            masked_features = features.clone()
            masked_features[mask] = 0  # 마스킹된 부분을 0으로 설정
            
            masked_x[node_type] = masked_features
            masks[node_type] = mask
        
        return masked_x, masks

    def forward(self, x_dict, edge_index_dict, edge_label_index=None,
                health_edge_index=None, health_scores=None, training=True, 
                pretraining=False):
        
        if pretraining:
            # Self-supervised pre-training mode
            masked_x, masks = self.mask_features(x_dict)
            current_x = {node_type: self.input_projections[node_type](features)
                        for node_type, features in masked_x.items()}
        else:
            # Downstream task mode
            current_x = {node_type: self.input_projections[node_type](features)
                        for node_type, features in x_dict.items()}
        
        # Encoding
        current_x = self.encoder(current_x, edge_index_dict)
        
        for node_type in current_x.keys():
            if node_type in self.batch_norms:
                current_x[node_type] = F.relu(self.batch_norms[node_type](current_x[node_type]))
        
        if pretraining:
            # Return reconstructions and contrastive features for pre-training
            reconstructions = {}
            for node_type in current_x.keys():
                reconstructions[node_type] = self.decoder[node_type](current_x[node_type])
            
            contrastive_features = {
                node_type: self.contrastive_projector(emb) 
                for node_type, emb in current_x.items()
            }
            
            return {
                'reconstructions': reconstructions,
                'contrastive_features': contrastive_features,
                'masks': masks,
                'original_features': x_dict
            }
        
        # Downstream task mode
        # Health-aware processing
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            food_embeddings = current_x['food']
            
            user_exp = user_embeddings.unsqueeze(0)
            food_exp = food_embeddings.unsqueeze(0)
            
            health_attn_out, _ = self.health_attention(user_exp, food_exp, food_exp)
            enhanced_user = health_attn_out.squeeze(0)
            
            user_indices_health = health_edge_index[0]
            food_indices_health = health_edge_index[1]
            
            health_weights = torch.sigmoid(health_scores * 2.0)
            health_updates = scatter(
                health_weights, food_indices_health, dim=0,
                dim_size=food_embeddings.size(0), reduce='mean'
            )
            
            health_scale = 0.2
            health_expanded = health_updates.unsqueeze(1).expand(-1, current_x['food'].size(1))
            current_x['food'] = current_x['food'] + health_scale * health_expanded
            current_x['user'] = enhanced_user
        
        # Task prediction
        if edge_label_index is not None:
            user_indices, food_indices = edge_label_index
            user_emb = current_x['user'][user_indices]
            food_emb = current_x['food'][food_indices]
            
            combined = torch.cat([user_emb, food_emb], dim=-1)
            return self.task_head(combined).squeeze()
        
        return current_x


### 3.6 Our Model

In [23]:
class FullHealthAwareGNN(nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, input_dims):
        super().__init__()

        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })

        self.conv = HeteroConv({
            ('user', 'eats', 'food'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False, edge_dim=1
            ),
            ('food', 'rev_eats', 'user'): GATConv(
                in_channels=(-1, -1), out_channels=out_channels,
                heads=4, concat=False, dropout=0.3, add_self_loops=False
            )
        })

        # 🌟 핵심 혁신: 건강 어텐션 메커니즘 🌟
        self.health_preference_net = nn.Sequential(
            nn.Linear(out_channels, out_channels // 2),
            nn.ReLU(),
            nn.Linear(out_channels // 2, 1),
            nn.Sigmoid()
        )

        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, 1),
            nn.Sigmoid()
        )

        self.batch_norms = nn.ModuleDict({
            node_type: nn.BatchNorm1d(out_channels) for node_type in metadata[0]
        })

    def forward(self, x_dict, edge_index_dict, edge_label_index,
                health_edge_index=None, health_scores=None, training=True):

        current_x = {node_type: self.input_projections[node_type](features)
                    for node_type, features in x_dict.items()}

        current_x = self.conv(current_x, edge_index_dict)

        for node_type in current_x.keys():
            if node_type in self.batch_norms:
                current_x[node_type] = F.relu(self.batch_norms[node_type](current_x[node_type]))

        # 🌟 완전한 건강 인식 처리 🌟
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            food_embeddings = current_x['food']

            # 개인화된 건강 선호도
            user_health_prefs = self.health_preference_net(user_embeddings)

            # 건강 정보 통합
            user_indices_health = health_edge_index[0]
            food_indices_health = health_edge_index[1]

            personal_weights = user_health_prefs[user_indices_health].squeeze()
            enhanced_health = torch.sigmoid(health_scores * 2.0)
            final_weights = personal_weights * enhanced_health

            health_updates = scatter(
                final_weights, food_indices_health, dim=0,
                dim_size=food_embeddings.size(0), reduce='mean'
            )

            # 건강 정보를 음식 표현에 통합
            health_scale = 0.2
            health_expanded = health_updates.unsqueeze(1).expand(-1, current_x['food'].size(1))
            current_x['food'] = current_x['food'] + health_scale * health_expanded

        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]

        combined = torch.cat([user_emb, food_emb], dim=-1)
        predictions = self.decoder(combined).squeeze()

        # 🌟 추론 시 건강한 선택 유도 🌟
        if not training and health_edge_index is not None:
            predictions = torch.sigmoid(torch.logit(predictions + 1e-7) + 0.1)

        return predictions

class GraphTransformerNetwork(nn.Module):
    """그래프 트랜스포머 네트워크 (GT/GraphiT 스타일)"""
    def __init__(self, hidden_channels, out_channels, metadata, input_dims, num_layers=3):
        super().__init__()
        
        self.num_layers = num_layers
        
        self.input_projections = nn.ModuleDict({
            node_type: nn.Linear(input_dims[node_type], hidden_channels)
            for node_type in metadata[0]
        })
        
        # Graph Transformer layers
        self.gt_layers = nn.ModuleList([
            GraphTransformerLayer(hidden_channels, num_heads=8)
            for _ in range(num_layers)
        ])
        
        # Edge encoding for different edge types
        self.edge_encoders = nn.ModuleDict({
            'eats': nn.Linear(1, hidden_channels),
            'healthness': nn.Linear(1, hidden_channels),
            'default': nn.Linear(1, hidden_channels)
        })
        
        # Health-aware graph attention
        self.health_graph_attn = nn.MultiheadAttention(
            embed_dim=hidden_channels,
            num_heads=4,
            dropout=0.1,
            batch_first=True
        )
        
        # Final projection to output dimension
        self.output_projection = nn.Linear(hidden_channels, out_channels)
        
        self.decoder = nn.Sequential(
            nn.Linear(out_channels * 2, hidden_channels),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_channels // 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x_dict, edge_index_dict, edge_label_index,
                health_edge_index=None, health_scores=None, training=True):
        
        # Input projection and combine all node types
        all_nodes = []
        node_type_map = {}
        offset = 0
        
        for node_type, features in x_dict.items():
            projected = self.input_projections[node_type](features)
            all_nodes.append(projected)
            node_type_map[node_type] = (offset, offset + features.size(0))
            offset += features.size(0)
        
        combined_nodes = torch.cat(all_nodes, dim=0)
        
        # Process through Graph Transformer layers
        for gt_layer in self.gt_layers:
            combined_nodes = gt_layer(combined_nodes, edge_index_dict, node_type_map)
        
        # Project to output dimension
        combined_nodes = self.output_projection(combined_nodes)
        
        # Split back to individual node types
        current_x = {}
        for node_type, (start, end) in node_type_map.items():
            current_x[node_type] = combined_nodes[start:end]
        
        # Health-aware processing
        if health_edge_index is not None and health_scores is not None:
            user_embeddings = current_x['user']
            food_embeddings = current_x['food']
            
            # Health-aware graph attention
            user_exp = user_embeddings.unsqueeze(0)
            food_exp = food_embeddings.unsqueeze(0)
            
            health_attn_out, _ = self.health_graph_attn(user_exp, food_exp, food_exp)
            enhanced_user = health_attn_out.squeeze(0)
            
            # Health integration
            user_indices_health = health_edge_index[0]
            food_indices_health = health_edge_index[1]
            
            health_weights = torch.sigmoid(health_scores * 2.5)
            health_updates = scatter(
                health_weights, food_indices_health, dim=0,
                dim_size=food_embeddings.size(0), reduce='mean'
            )
            
            health_scale = 0.35
            health_expanded = health_updates.unsqueeze(1).expand(-1, current_x['food'].size(1))
            current_x['food'] = current_x['food'] + health_scale * health_expanded
            current_x['user'] = enhanced_user
        
        user_indices, food_indices = edge_label_index
        user_emb = current_x['user'][user_indices]
        food_emb = current_x['food'][food_indices]
        
        combined = torch.cat([user_emb, food_emb], dim=-1)
        return self.decoder(combined).squeeze()

class GraphTransformerLayer(nn.Module):
    """단일 그래프 트랜스포머 레이어"""
    def __init__(self, hidden_dim, num_heads=8, dropout=0.1):
        super().__init__()
        
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        self.feed_forward = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 4, hidden_dim),
            nn.Dropout(dropout)
        )
        
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index_dict, node_type_map):
        # Self-attention with residual connection
        x_norm = self.norm1(x)
        x_expanded = x_norm.unsqueeze(0)
        attn_out, _ = self.attention(x_expanded, x_expanded, x_expanded)
        x = x + self.dropout(attn_out.squeeze(0))
        
        # Feed forward with residual connection
        x_norm = self.norm2(x)
        ff_out = self.feed_forward(x_norm)
        x = x + self.dropout(ff_out)
        
        return x


### 3.6 Loss Functions

In [24]:
class StandardLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCELoss()

    def forward(self, predictions, targets, **kwargs):
        return self.bce(predictions, targets)

class HealthAwareLoss(nn.Module):
    def __init__(self, health_lambda=0.1):
        super().__init__()
        self.health_lambda = health_lambda
        self.bce = nn.BCELoss()

    def forward(self, predictions, targets, health_scores=None):
        base_loss = self.bce(predictions, targets)

        if health_scores is not None and len(health_scores) > 0:
            # 건강한 음식일수록 높은 예측값을 가지도록 정규화
            health_normalized = torch.sigmoid(health_scores)
            health_reg = -torch.mean(predictions * health_normalized.mean())
            total_loss = base_loss + self.health_lambda * health_reg
        else:
            total_loss = base_loss

        return total_loss

class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature

    def forward(self, predictions, targets, user_proj=None, food_proj=None, **kwargs):
        base_loss = F.binary_cross_entropy(predictions, targets)
        
        if user_proj is not None and food_proj is not None:
            # InfoNCE contrastive loss
            user_proj = F.normalize(user_proj, dim=-1)
            food_proj = F.normalize(food_proj, dim=-1)
            
            similarity = torch.matmul(user_proj, food_proj.T) / self.temperature
            labels = torch.arange(similarity.size(0), device=similarity.device)
            
            contrastive_loss = F.cross_entropy(similarity, labels)
            return base_loss + 0.1 * contrastive_loss
        
        return base_loss

class MultiTaskLoss(nn.Module):
    def __init__(self, task_weights=None):
        super().__init__()
        self.task_weights = task_weights or {'preference': 1.0, 'health_score': 0.5, 
                                           'nutrition_category': 0.3, 'health_risk': 0.2}
        self.bce = nn.BCELoss()
        self.mse = nn.MSELoss()
        self.ce = nn.CrossEntropyLoss()

    def forward(self, predictions, targets, **kwargs):
        if isinstance(predictions, dict):
            # Multi-task mode
            total_loss = 0
            
            # Task 1: Preference prediction
            if 'preference' in predictions and 'preference' in targets:
                pref_loss = self.bce(predictions['preference'], targets['preference'])
                total_loss += self.task_weights['preference'] * pref_loss
            
            # Task 2: Health score prediction  
            if 'health_score' in predictions and 'health_score' in targets:
                health_loss = self.mse(predictions['health_score'], targets['health_score'])
                total_loss += self.task_weights['health_score'] * health_loss
            
            # Task 3: Nutrition classification
            if 'nutrition_category' in predictions and 'nutrition_category' in targets:
                nutrition_loss = self.ce(predictions['nutrition_category'], targets['nutrition_category'])
                total_loss += self.task_weights['nutrition_category'] * nutrition_loss
            
            # Task 4: Health risk assessment
            if 'health_risk' in predictions and 'health_risk' in targets:
                risk_loss = self.ce(predictions['health_risk'], targets['health_risk'])
                total_loss += self.task_weights['health_risk'] * risk_loss
            
            return total_loss
        else:
            # Single task mode
            return self.bce(predictions, targets)

class SelfSupervisedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha  # Balance between reconstruction and contrastive
        self.mse = nn.MSELoss()

    def forward(self, predictions, targets=None, **kwargs):
        if isinstance(predictions, dict) and 'reconstructions' in predictions:
            # Self-supervised pre-training mode
            reconstructions = predictions['reconstructions']
            original_features = predictions['original_features']
            masks = predictions['masks']
            contrastive_features = predictions['contrastive_features']
            
            # Reconstruction loss (only on masked parts)
            recon_loss = 0
            for node_type in reconstructions.keys():
                mask = masks[node_type]
                recon = reconstructions[node_type]
                original = original_features[node_type]
                
                masked_recon_loss = self.mse(recon[mask], original[mask])
                recon_loss += masked_recon_loss
            
            # Contrastive loss between different node types
            contrastive_loss = 0
            if 'user' in contrastive_features and 'food' in contrastive_features:
                user_feat = F.normalize(contrastive_features['user'], dim=-1)
                food_feat = F.normalize(contrastive_features['food'], dim=-1)
                
                # Simple contrastive loss
                similarity = torch.matmul(user_feat, food_feat.T)
                positive_pairs = torch.eye(min(user_feat.size(0), food_feat.size(0)), 
                                         device=user_feat.device)
                contrastive_loss = -torch.sum(positive_pairs * similarity) / positive_pairs.sum()
            
            return self.alpha * recon_loss + (1 - self.alpha) * contrastive_loss
        else:
            # Standard supervised loss
            return F.binary_cross_entropy(predictions, targets)

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, predictions, targets, **kwargs):
        ce_loss = F.binary_cross_entropy(predictions, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

class AdvancedHealthAwareLoss(nn.Module):
    def __init__(self, health_lambda=0.1, diversity_lambda=0.05, consistency_lambda=0.02):
        super().__init__()
        self.health_lambda = health_lambda
        self.diversity_lambda = diversity_lambda
        self.consistency_lambda = consistency_lambda
        self.bce = nn.BCELoss()

    def forward(self, predictions, targets, health_scores=None, user_embeddings=None, **kwargs):
        base_loss = self.bce(predictions, targets)
        
        total_loss = base_loss
        
        # Health regularization
        if health_scores is not None and len(health_scores) > 0:
            health_normalized = torch.sigmoid(health_scores)
            avg_health = health_normalized.mean()
            
            # Encourage healthy food recommendations
            health_reg = -torch.mean(predictions * avg_health)
            total_loss += self.health_lambda * health_reg
        
        # Diversity regularization (encourage diverse recommendations)
        if user_embeddings is not None:
            # Compute pairwise similarities
            normalized_embeddings = F.normalize(user_embeddings, dim=-1)
            similarity_matrix = torch.matmul(normalized_embeddings, normalized_embeddings.T)
            
            # Penalize too similar embeddings (encourage diversity)
            diversity_loss = torch.mean(torch.triu(similarity_matrix, diagonal=1) ** 2)
            total_loss += self.diversity_lambda * diversity_loss
        
        # Consistency regularization (predictions should be consistent with health scores)
        if health_scores is not None and len(health_scores) > 0:
            health_normalized = torch.sigmoid(health_scores * 2.0)
            avg_health = health_normalized.mean()
            
            # Consistency between predictions and health
            pred_health_consistency = torch.abs(predictions.mean() - avg_health)
            total_loss += self.consistency_lambda * pred_health_consistency
        
        return total_loss


## 4. 실험 실행 함수들

In [ ]:
def run_traditional_ml_models(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    models = {
        'Logistic_Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Random_Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Gradient_Boosting': GradientBoostingClassifier(random_state=42),
        #'XGBoost': xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
        'SVM': SVC(probability=True, random_state=42, kernel='rbf')
    }

    results = {}

    for name, model in models.items():
        print(f"   Training {name}...")
        start_time = time.time()

        model.fit(X_train, y_train)

        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_pred_proba = model.decision_function(X_test)
            # Normalize to [0,1]
            y_pred_proba = (y_pred_proba - y_pred_proba.min()) / (y_pred_proba.max() - y_pred_proba.min())
        
        y_pred = model.predict(X_test)
        training_time = time.time() - start_time

        results[name] = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'auc': roc_auc_score(y_test, y_pred_proba),
            'training_time': training_time
        }

        print(f"     {name}: F1={results[name]['f1']:.3f}, AUC={results[name]['auc']:.3f}")

    return results

def run_deep_learning_models(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    models = {
        'Simple_MLP': SimpleMLP(X.shape[1]),
        'Advanced_MLP': AdvancedMLP(X.shape[1])
    }

    results = {}

    for name, model in models.items():
        print(f"   Training {name}...")
        start_time = time.time()

        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.BCELoss()

        X_train_tensor = torch.FloatTensor(X_train).to(device)
        y_train_tensor = torch.FloatTensor(y_train).to(device)
        X_test_tensor = torch.FloatTensor(X_test).to(device)

        # 훈련
        model.train()
        for epoch in range(50):
            optimizer.zero_grad()
            outputs = model(X_train_tensor)
            loss = criterion(outputs.squeeze(), y_train_tensor)
            loss.backward()
            optimizer.step()

        # 평가
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
            y_pred_proba = test_outputs.squeeze().cpu().numpy()
            y_pred = (y_pred_proba > 0.5).astype(int)

        training_time = time.time() - start_time

        results[name] = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'auc': roc_auc_score(y_test, y_pred_proba),
            'training_time': training_time
        }

        print(f"     {name}: F1={results[name]['f1']:.3f}, AUC={results[name]['auc']:.3f}")

    return results

def train_gnn_model(model, data, sample_indices, threshold, use_health_loss=True):
    start_time = time.time()

    # 데이터 준비
    eats_edge_index = data[('user', 'eats', 'food')].edge_index
    eats_edge_attr = data[('user', 'eats', 'food')].edge_attr

    user_indices = eats_edge_index[0].cpu().numpy()
    food_indices = eats_edge_index[1].cpu().numpy()
    eats_scores = eats_edge_attr.cpu().numpy()

    user_indices_sample = user_indices[sample_indices]
    food_indices_sample = food_indices[sample_indices]
    y_sample = (eats_scores[sample_indices] > threshold).astype(int)

    # Train-test split
    indices = np.arange(len(y_sample))
    indices_train, indices_test = train_test_split(
        indices, test_size=0.2, random_state=42, stratify=y_sample
    )

    # 모델 설정
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    if use_health_loss:
        criterion = HealthAwareLoss(health_lambda=0.1)
    else:
        criterion = StandardLoss()

    # 데이터 텐서
    x_dict = {k: v.to(device) for k, v in data.x_dict.items()}
    edge_index_dict = {
        ('user', 'eats', 'food'): data[('user', 'eats', 'food')].edge_index.to(device),
        ('food', 'rev_eats', 'user'): torch.stack([
            data[('user', 'eats', 'food')].edge_index[1],
            data[('user', 'eats', 'food')].edge_index[0]
        ]).to(device)
    }

    health_edge_index = data[('user', 'healthness', 'food')].edge_index.to(device)
    health_scores_tensor = data[('user', 'healthness', 'food')].edge_attr.to(device)

    # 훈련
    model.train()
    for epoch in range(40):
        optimizer.zero_grad()

        train_edge_index = torch.stack([
            torch.tensor(user_indices_sample[indices_train], dtype=torch.long),
            torch.tensor(food_indices_sample[indices_train], dtype=torch.long)
        ]).to(device)
        train_targets = torch.tensor(y_sample[indices_train], dtype=torch.float).to(device)

        predictions = model(
            x_dict, edge_index_dict, train_edge_index,
            health_edge_index=health_edge_index,
            health_scores=health_scores_tensor,
            training=True
        )

        if use_health_loss:
            loss = criterion(predictions, train_targets, health_scores_tensor)
        else:
            loss = criterion(predictions, train_targets)

        loss.backward()
        optimizer.step()

    # 평가
    model.eval()
    test_edge_index = torch.stack([
        torch.tensor(user_indices_sample[indices_test], dtype=torch.long),
        torch.tensor(food_indices_sample[indices_test], dtype=torch.long)
    ]).to(device)

    with torch.no_grad():
        test_predictions = model(
            x_dict, edge_index_dict, test_edge_index,
            health_edge_index=health_edge_index,
            health_scores=health_scores_tensor,
            training=False
        )

        y_pred_proba = test_predictions.cpu().numpy()
        y_pred = (y_pred_proba > 0.5).astype(int)

    y_test = y_sample[indices_test]
    training_time = time.time() - start_time

    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_pred_proba),
        'training_time': training_time
    }

def train_enhanced_gnn_model(model, data, sample_indices, threshold, model_type='enhanced'):
    start_time = time.time()

    # 데이터 준비
    eats_edge_index = data[('user', 'eats', 'food')].edge_index
    eats_edge_attr = data[('user', 'eats', 'food')].edge_attr

    user_indices = eats_edge_index[0].cpu().numpy()
    food_indices = eats_edge_index[1].cpu().numpy()
    eats_scores = eats_edge_attr.cpu().numpy()

    user_indices_sample = user_indices[sample_indices]
    food_indices_sample = food_indices[sample_indices]
    y_sample = (eats_scores[sample_indices] > threshold).astype(int)

    # Train-test split
    indices = np.arange(len(y_sample))
    indices_train, indices_test = train_test_split(
        indices, test_size=0.2, random_state=42, stratify=y_sample
    )

    # 모델 설정
    model = model.to(device)
    
    # 고급 옵티마이저 (AdamW + 학습률 스케줄링)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
    
    # 손실 함수 선택
    if model_type == 'contrastive':
        criterion = ContrastiveLoss(temperature=0.07)
    elif model_type == 'multitask':
        criterion = MultiTaskLoss()
    elif model_type == 'self_supervised':
        criterion = SelfSupervisedLoss(alpha=0.6)
    elif model_type == 'focal':
        criterion = FocalLoss(alpha=1, gamma=2)
    else:
        criterion = AdvancedHealthAwareLoss(health_lambda=0.15, diversity_lambda=0.05)

    # 데이터 텐서
    x_dict = {k: v.to(device) for k, v in data.x_dict.items()}
    edge_index_dict = {
        ('user', 'eats', 'food'): data[('user', 'eats', 'food')].edge_index.to(device),
        ('food', 'rev_eats', 'user'): torch.stack([
            data[('user', 'eats', 'food')].edge_index[1],
            data[('user', 'eats', 'food')].edge_index[0]
        ]).to(device)
    }
    
    health_edge_index = data[('user', 'healthness', 'food')].edge_index.to(device)
    health_scores_tensor = data[('user', 'healthness', 'food')].edge_attr.to(device)

    # Early stopping
    best_loss = float('inf')
    patience = 10
    patience_counter = 0

    # 훈련 (더 많은 에포크로 개선)
    model.train()
    for epoch in range(60):  # 증가된 에포크
        optimizer.zero_grad()

        train_edge_index = torch.stack([
            torch.tensor(user_indices_sample[indices_train], dtype=torch.long),
            torch.tensor(food_indices_sample[indices_train], dtype=torch.long)
        ]).to(device)
        train_targets = torch.tensor(y_sample[indices_train], dtype=torch.float).to(device)

        # 모델별 특별 처리
        if model_type == 'multitask':
            predictions = model(
                x_dict, edge_index_dict, train_edge_index,
                health_edge_index=health_edge_index,
                health_scores=health_scores_tensor,
                training=True,
                return_all_tasks=True
            )
            
            # 멀티태스크 타겟 생성 (간단한 예시)
            targets_dict = {
                'preference': train_targets,
                'health_score': health_scores_tensor[:len(train_targets)],  # 예시
                'nutrition_category': torch.randint(0, 5, (len(train_targets),), device=device),  # 예시
                'health_risk': torch.randint(0, 3, (len(train_targets),), device=device)  # 예시
            }
            
            loss = criterion(predictions, targets_dict)
            
        elif model_type == 'contrastive':
            predictions = model(
                x_dict, edge_index_dict, train_edge_index,
                health_edge_index=health_edge_index,
                health_scores=health_scores_tensor,
                training=True
            )
            
            # Contrastive projections 사용
            user_proj = getattr(model, 'user_projections', None)
            food_proj = getattr(model, 'food_projections', None)
            
            loss = criterion(predictions, train_targets, 
                           user_proj=user_proj, food_proj=food_proj)
            
        else:
            predictions = model(
                x_dict, edge_index_dict, train_edge_index,
                health_edge_index=health_edge_index,
                health_scores=health_scores_tensor,
                training=True
            )
            
            # 사용자 임베딩 추출 (diversity regularization용)
            user_embeddings = None
            if hasattr(model, 'get_user_embeddings'):
                user_embeddings = model.get_user_embeddings(x_dict, edge_index_dict)
            
            loss = criterion(predictions, train_targets, 
                           health_scores=health_scores_tensor,
                           user_embeddings=user_embeddings)

        loss.backward()
        
        # Gradient clipping (안정성 향상)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        scheduler.step()

        # Early stopping 체크
        if loss.item() < best_loss:
            best_loss = loss.item()
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"      Early stopping at epoch {epoch+1}")
            break

    # 평가
    model.eval()
    test_edge_index = torch.stack([
        torch.tensor(user_indices_sample[indices_test], dtype=torch.long),
        torch.tensor(food_indices_sample[indices_test], dtype=torch.long)
    ]).to(device)

    with torch.no_grad():
        if model_type == 'multitask':
            test_predictions_dict = model(
                x_dict, edge_index_dict, test_edge_index,
                health_edge_index=health_edge_index,
                health_scores=health_scores_tensor,
                training=False,
                return_all_tasks=True
            )
            test_predictions = test_predictions_dict['preference']
        else:
            test_predictions = model(
                x_dict, edge_index_dict, test_edge_index,
                health_edge_index=health_edge_index,
                health_scores=health_scores_tensor,
                training=False
            )

        y_pred_proba = test_predictions.cpu().numpy()
        y_pred = (y_pred_proba > 0.5).astype(int)

    y_test = y_sample[indices_test]
    training_time = time.time() - start_time

    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_pred_proba),
        'training_time': training_time,
        'final_loss': best_loss
    }



## 5. 종합 실험 실행

### 5.1 데이터 준비 및 설정

In [26]:
# 실험 설정
SAMPLE_SIZE = 50000  # 실험 샘플 크기
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

print("🔬 Health-Aware GNN Comprehensive Experiments 🔬")
print("=" * 70)
print(f"📊 실험 설정:")
print(f"   샘플 크기: {SAMPLE_SIZE:,}")
print(f"   Random Seed: {RANDOM_SEED}")
print(f"   디바이스: {device}")

# ML 데이터 준비
print("\n📊 ML 데이터 준비 중...")
X, y, threshold, sample_indices = prepare_ml_data(data, SAMPLE_SIZE)

print(f"✅ 데이터 준비 완료:")
print(f"   특성 차원: {X.shape[1]}")
print(f"   샘플 수: {len(y):,}")
print(f"   임계값 (중앙값): {threshold:.3f}")
print(f"   Positive 비율: {y.mean():.3f}")

# GNN 메타데이터 준비
metadata = (list(data.x_dict.keys()), list(data.edge_index_dict.keys()))
input_dims = {node_type: features.shape[1] for node_type, features in data.x_dict.items()}

print(f"\n📊 GNN 메타데이터:")
print(f"   노드 타입: {metadata[0]}")
print(f"   엣지 타입 수: {len(metadata[1])}")
for node_type, dim in input_dims.items():
    print(f"   {node_type} 특성 차원: {dim}")

# 결과 저장용 딕셔너리
all_results = {}

🔬 Health-Aware GNN Comprehensive Experiments 🔬
📊 실험 설정:
   샘플 크기: 50,000
   Random Seed: 42
   디바이스: cuda

📊 ML 데이터 준비 중...
✅ 데이터 준비 완료:
   특성 차원: 88
   샘플 수: 50,000
   임계값 (중앙값): 0.667
   Positive 비율: 0.492

📊 GNN 메타데이터:
   노드 타입: ['user', 'food', 'ingredient', 'time']
   엣지 타입 수: 9
   user 특성 차원: 29
   food 특성 차원: 17
   ingredient 특성 차원: 101
   time 특성 차원: 4


### 5.2 Traditional ML Models 실험

In [45]:
print("\n🔹 1. TRADITIONAL ML MODELS")
print("-" * 50)

ml_results = run_traditional_ml_models(X, y)
all_results['traditional_ml'] = ml_results

print("\nTraditional ML Results:")
for model_name, metrics in ml_results.items():
    print(f"   {model_name:<20}: F1={metrics['f1']:.3f}, AUC={metrics['auc']:.3f}, Time={metrics['training_time']:.1f}s")


🔹 1. TRADITIONAL ML MODELS
--------------------------------------------------
   Training Logistic_Regression...
     Logistic_Regression: F1=0.614, AUC=0.692
   Training Random_Forest...
     Random_Forest: F1=0.759, AUC=0.850
   Training Gradient_Boosting...
     Gradient_Boosting: F1=0.733, AUC=0.831
   Training XGBoost...
     XGBoost: F1=0.761, AUC=0.851
   Training SVM...
     SVM: F1=0.501, AUC=0.579

Traditional ML Results:
   Logistic_Regression : F1=0.614, AUC=0.692, Time=4.6s
   Random_Forest       : F1=0.759, AUC=0.850, Time=17.2s
   Gradient_Boosting   : F1=0.733, AUC=0.831, Time=45.8s
   XGBoost             : F1=0.761, AUC=0.851, Time=0.6s
   SVM                 : F1=0.501, AUC=0.579, Time=2111.5s


### 5.3 Deep Learning Models 실험

In [28]:
print("\n🔹 2. DEEP LEARNING MODELS")
print("-" * 50)

dl_results = run_deep_learning_models(X, y)
all_results['deep_learning'] = dl_results

print("\nDeep Learning Results:")
for model_name, metrics in dl_results.items():
    print(f"   {model_name:<20}: F1={metrics['f1']:.3f}, AUC={metrics['auc']:.3f}, Time={metrics['training_time']:.1f}s")


🔹 2. DEEP LEARNING MODELS
--------------------------------------------------
   Training Simple_MLP...
     Simple_MLP: F1=0.660, AUC=0.452
   Training Advanced_MLP...
     Advanced_MLP: F1=0.505, AUC=0.606

Deep Learning Results:
   Simple_MLP          : F1=0.660, AUC=0.452, Time=4.8s
   Advanced_MLP        : F1=0.505, AUC=0.606, Time=0.8s


### 5.4 GNN Baseline Models 실험

In [29]:
print("\n🔹 3. GNN BASELINE MODELS (No Health Awareness)")
print("-" * 50)

gnn_models = {
    'Vanilla_GCN': VanillaGNN(128, 64, metadata, input_dims),
    'GraphSAGE': GraphSAGE(128, 64, metadata, input_dims),
    'GAT_NoHealth': GAT_NoHealth(128, 64, metadata, input_dims)
}

gnn_results = {}

for name, model in gnn_models.items():
    print(f"   Training {name}...")
    result = train_gnn_model(model, data, sample_indices, threshold, use_health_loss=False)
    gnn_results[name] = result
    print(f"     {name}: F1={result['f1']:.3f}, AUC={result['auc']:.3f}, Time={result['training_time']:.1f}s")

all_results['gnn_baselines'] = gnn_results

print("\n📊 GNN Baseline Results:")
for model_name, metrics in gnn_results.items():
    print(f"   {model_name:<20}: F1={metrics['f1']:.3f}, AUC={metrics['auc']:.3f}, Time={metrics['training_time']:.1f}s")


🔹 3. GNN BASELINE MODELS (No Health Awareness)
--------------------------------------------------
   Training Vanilla_GCN...
     Vanilla_GCN: F1=0.000, AUC=0.500, Time=3.3s
   Training GraphSAGE...
     GraphSAGE: F1=0.660, AUC=0.500, Time=0.7s
   Training GAT_NoHealth...
     GAT_NoHealth: F1=0.211, AUC=0.537, Time=1.7s

📊 GNN Baseline Results:
   Vanilla_GCN         : F1=0.000, AUC=0.500, Time=3.3s
   GraphSAGE           : F1=0.660, AUC=0.500, Time=0.7s
   GAT_NoHealth        : F1=0.211, AUC=0.537, Time=1.7s


### 5.5 Ablation Study Models 실험

In [30]:
print("\n🔹 4. ABLATION STUDY MODELS")
print("-" * 50)

ablation_models = {
    'Health_NoAttention': HealthAware_NoAttention(128, 64, metadata, input_dims),
    'Health_NoLoss': HealthAware_NoLoss(128, 64, metadata, input_dims)
}

ablation_results = {}

for name, model in ablation_models.items():
    print(f"   Training {name}...")
    if 'NoLoss' in name:
        result = train_gnn_model(model, data, sample_indices, threshold, use_health_loss=False)
    else:
        result = train_gnn_model(model, data, sample_indices, threshold, use_health_loss=True)
    ablation_results[name] = result
    print(f"     {name}: F1={result['f1']:.3f}, AUC={result['auc']:.3f}, Time={result['training_time']:.1f}s")

all_results['ablation_study'] = ablation_results

print("\n📊 Ablation Study Results:")
for model_name, metrics in ablation_results.items():
    print(f"   {model_name:<20}: F1={metrics['f1']:.3f}, AUC={metrics['auc']:.3f}, Time={metrics['training_time']:.1f}s")


🔹 4. ABLATION STUDY MODELS
--------------------------------------------------
   Training Health_NoAttention...
     Health_NoAttention: F1=0.660, AUC=0.500, Time=0.5s
   Training Health_NoLoss...
     Health_NoLoss: F1=0.121, AUC=0.519, Time=1.5s

📊 Ablation Study Results:
   Health_NoAttention  : F1=0.660, AUC=0.500, Time=0.5s
   Health_NoLoss       : F1=0.121, AUC=0.519, Time=1.5s


### 5.6 Our Full Health-Aware Model 실험

In [46]:
print("\n🔹 5. SOTA MODELS (수정됨)")
print("-" * 50)

# SOTA 모델들 정의 (수정된 TransformerEnhanced_GNN 사용)
sota_models = {
    'TransformerEnhanced_GNN': TransformerEnhancedGNN_Fixed(128, 64, metadata, input_dims),
    'ContrastiveLearning_GNN': ContrastiveLearningGNN(128, 64, metadata, input_dims), 
    'MetaLearning_GNN': MetaLearningGNN(128, 64, metadata, input_dims),
    'GraphTransformer_Net': GraphTransformerNetwork(128, 64, metadata, input_dims),
    'MultiTaskHealth_GNN': MultiTaskHealthGNN(128, 64, metadata, input_dims),
    'SelfSupervised_GNN': SelfSupervisedGNN(128, 64, metadata, input_dims)
}

sota_results = {}

for name, model in sota_models.items():
    print(f"   Training {name}...")
    try:
        result = train_enhanced_gnn_model(model, data, sample_indices, threshold)
        sota_results[name] = result
        print(f"     {name}: F1={result['f1']:.3f}, AUC={result['auc']:.3f}, Time={result['training_time']:.1f}s")
    except Exception as e:
        print(f"     {name}: ERROR - {str(e)}")
        # 에러가 발생한 모델은 기본값으로 저장
        sota_results[name] = {
            'f1': 0.000, 'auc': 0.500, 'accuracy': 0.000, 
            'precision': 0.000, 'recall': 0.000, 'training_time': 0.0
        }

all_results['enhanced_models'] = sota_results

print("\n📊 SOTA Models Results:")
for model_name, metrics in sota_results.items():
    print(f"   {model_name:<25}: F1={metrics['f1']:.3f}, AUC={metrics['auc']:.3f}, Time={metrics['training_time']:.1f}s")


🔹 5. SOTA MODELS (수정됨)
--------------------------------------------------
   Training TransformerEnhanced_GNN...
     TransformerEnhanced_GNN: ERROR - CUDA out of memory. Tried to allocate 40.73 GiB (GPU 0; 8.00 GiB total capacity; 868.61 MiB already allocated; 5.24 GiB free; 994.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
   Training ContrastiveLearning_GNN...
      Early stopping at epoch 35
     ContrastiveLearning_GNN: F1=0.658, AUC=0.532, Time=1.4s
   Training MetaLearning_GNN...
     MetaLearning_GNN: ERROR - CUDA out of memory. Tried to allocate 235.94 GiB (GPU 0; 8.00 GiB total capacity; 1.08 GiB already allocated; 4.99 GiB free; 1.22 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_AL

In [32]:
print("\n🔹 6. OUR FULL HEALTH-AWARE MODEL")
print("-" * 50)

our_model = FullHealthAwareGNN(128, 64, metadata, input_dims)

print("   Training Full Health-Aware GNN...")
our_result = train_gnn_model(our_model, data, sample_indices, threshold, use_health_loss=True)

our_results = {'Full_HealthAware_GNN': our_result}
all_results['our_model'] = our_results

print(f"     Full_HealthAware_GNN: F1={our_result['f1']:.3f}, AUC={our_result['auc']:.3f}, Time={our_result['training_time']:.1f}s")

print("\n🌟 Our Model Results:")
print(f"   Full_HealthAware_GNN : F1={our_result['f1']:.3f}, AUC={our_result['auc']:.3f}, Time={our_result['training_time']:.1f}s")


🔹 6. OUR FULL HEALTH-AWARE MODEL
--------------------------------------------------
   Training Full Health-Aware GNN...
     Full_HealthAware_GNN: F1=0.659, AUC=0.528, Time=2.0s

🌟 Our Model Results:
   Full_HealthAware_GNN : F1=0.659, AUC=0.528, Time=2.0s


## 6. Health Score Analysis

In [33]:
print("\n🔹 7. HEALTH SCORE ANALYSIS")
print("-" * 50)

def analyze_health_scores(data, sample_indices):
    """건강 점수 분석"""

    health_edge_index = data[('user', 'healthness', 'food')].edge_index
    health_scores = data[('user', 'healthness', 'food')].edge_attr

    eats_edge_index = data[('user', 'eats', 'food')].edge_index
    eats_scores = data[('user', 'eats', 'food')].edge_attr

    print("   Analyzing health scores...")

    # 건강 점수 통계
    health_stats = {
        'mean': health_scores.mean().item(),
        'std': health_scores.std().item(),
        'min': health_scores.min().item(),
        'max': health_scores.max().item(),
        'median': health_scores.median().item()
    }

    # 선호도-건강도 상관관계
    eats_sample = eats_scores[sample_indices].cpu().numpy()

    # 샘플된 eats 엣지에 대응하는 건강 점수 찾기
    user_indices_sample = eats_edge_index[0][sample_indices].cpu().numpy()
    food_indices_sample = eats_edge_index[1][sample_indices].cpu().numpy()

    health_user_indices = health_edge_index[0].cpu().numpy()
    health_food_indices = health_edge_index[1].cpu().numpy()
    health_scores_array = health_scores.cpu().numpy()

    # 매칭되는 건강 점수 찾기
    matched_health_scores = []
    for u, f in zip(user_indices_sample, food_indices_sample):
        mask = (health_user_indices == u) & (health_food_indices == f)
        if mask.any():
            matched_health_scores.append(health_scores_array[mask][0])
        else:
            matched_health_scores.append(health_stats['mean'])  # 평균값으로 대체

    matched_health_scores = np.array(matched_health_scores)

    # 상관관계 계산
    correlation = np.corrcoef(eats_sample, matched_health_scores)[0, 1]

    print(f"     Health score statistics:")
    print(f"       Mean: {health_stats['mean']:.3f}")
    print(f"       Std:  {health_stats['std']:.3f}")
    print(f"       Range: [{health_stats['min']:.3f}, {health_stats['max']:.3f}]")
    print(f"       Median: {health_stats['median']:.3f}")
    print(f"     Health-Preference correlation: {correlation:.3f}")

    return {
        'health_stats': health_stats,
        'preference_health_correlation': correlation,
        'n_health_edges': len(health_scores),
        'n_eats_edges': len(eats_scores),
        'matched_health_scores': matched_health_scores
    }

health_analysis = analyze_health_scores(data, sample_indices)
all_results['health_analysis'] = health_analysis


🔹 7. HEALTH SCORE ANALYSIS
--------------------------------------------------
   Analyzing health scores...
     Health score statistics:
       Mean: 0.657
       Std:  0.081
       Range: [0.295, 0.958]
       Median: 0.650
     Health-Preference correlation: 0.046


## 7. 종합 결과 분석 및 시각화

In [34]:
print("\n" + "=" * 80)
print("🏆 COMPREHENSIVE COMPARISON RESULTS (SOTA Enhanced)")
print("=" * 80)

# 모든 모델 결과 수집 (SOTA 모델 포함)
comparison_results = []

# Traditional ML
for model_name, metrics in all_results['traditional_ml'].items():
    comparison_results.append({
        'category': 'Traditional ML',
        'model': model_name,
        'f1': metrics['f1'],
        'auc': metrics['auc'],
        'accuracy': metrics['accuracy'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'training_time': metrics['training_time']
    })

# Deep Learning
for model_name, metrics in all_results['deep_learning'].items():
    comparison_results.append({
        'category': 'Deep Learning',
        'model': model_name,
        'f1': metrics['f1'],
        'auc': metrics['auc'],
        'accuracy': metrics['accuracy'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'training_time': metrics['training_time']
    })

# GNN Baselines
for model_name, metrics in all_results['gnn_baselines'].items():
    comparison_results.append({
        'category': 'GNN Baseline',
        'model': model_name,
        'f1': metrics['f1'],
        'auc': metrics['auc'],
        'accuracy': metrics['accuracy'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'training_time': metrics['training_time']
    })

# Ablation Study
for model_name, metrics in all_results['ablation_study'].items():
    comparison_results.append({
        'category': 'Ablation Study',
        'model': model_name,
        'f1': metrics['f1'],
        'auc': metrics['auc'],
        'accuracy': metrics['accuracy'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'training_time': metrics['training_time']
    })

# Our Original Model
for model_name, metrics in all_results['our_model'].items():
    comparison_results.append({
        'category': 'Our Original',
        'model': model_name,
        'f1': metrics['f1'],
        'auc': metrics['auc'],
        'accuracy': metrics['accuracy'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'training_time': metrics['training_time']
    })

# 🚀 SOTA Enhanced Models
for model_name, metrics in all_results['enhanced_models'].items():
    comparison_results.append({
        'category': '🚀 SOTA Enhanced',
        'model': model_name,
        'f1': metrics['f1'],
        'auc': metrics['auc'],
        'accuracy': metrics['accuracy'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'training_time': metrics['training_time']
    })

# DataFrame으로 변환
results_df = pd.DataFrame(comparison_results)

print("📊 Performance Ranking (by F1 Score):")
print("-" * 120)
print(f"{'Rank':<4} {'Category':<20} {'Model':<30} {'F1':<8} {'AUC':<8} {'Acc':<8} {'Prec':<8} {'Rec':<8} {'Time(s)':<8}")
print("-" * 120)

# F1 점수로 정렬
results_df_sorted = results_df.sort_values('f1', ascending=False).reset_index(drop=True)

for i, row in results_df_sorted.iterrows():
    rank = i + 1
    category_display = row['category'][:19]  # 길이 제한
    model_display = row['model'][:29]  # 길이 제한
    
    # SOTA 모델 하이라이트
    if '🚀' in row['category']:
        print(f"🌟{rank:<3} {category_display:<20} {model_display:<30} "
              f"{row['f1']:<8.3f} {row['auc']:<8.3f} {row['accuracy']:<8.3f} "
              f"{row['precision']:<8.3f} {row['recall']:<8.3f} {row['training_time']:<8.1f}")
    else:
        print(f"{rank:<4} {category_display:<20} {model_display:<30} "
              f"{row['f1']:<8.3f} {row['auc']:<8.3f} {row['accuracy']:<8.3f} "
              f"{row['precision']:<8.3f} {row['recall']:<8.3f} {row['training_time']:<8.1f}")

# 최고 성능 모델들 식별
best_overall = results_df_sorted.iloc[0]
best_sota = results_df[results_df['category'] == '🚀 SOTA Enhanced'].sort_values('f1', ascending=False).iloc[0] if len(results_df[results_df['category'] == '🚀 SOTA Enhanced']) > 0 else None

print("-" * 120)
print(f"🏆 BEST OVERALL: {best_overall['model']} ({best_overall['category']}) - F1: {best_overall['f1']:.3f}")
if best_sota is not None:
    print(f"🚀 BEST SOTA MODEL: {best_sota['model']} - F1: {best_sota['f1']:.3f}")

# 카테고리별 최고 성능
print(f"\n🥇 Best in Each Category:")
category_best = results_df.loc[results_df.groupby('category')['f1'].idxmax()]
for _, row in category_best.iterrows():
    print(f"   {row['category']:<20}: {row['model']:<30} (F1: {row['f1']:.3f})")

# 성능 개선 분석
original_best = all_results['our_model']['Full_HealthAware_GNN']['f1']
if best_sota is not None:
    sota_best = best_sota['f1']
    improvement = sota_best - original_best
    improvement_pct = (improvement / original_best) * 100
    
    print(f"\n📈 PERFORMANCE IMPROVEMENT:")
    print(f"   Original Best Model: {original_best:.3f}")
    print(f"   SOTA Enhanced Best: {sota_best:.3f}")
    print(f"   Absolute Improvement: {improvement:+.3f}")
    print(f"   Relative Improvement: {improvement_pct:+.1f}%")

# Top 5 모델들
print(f"\n🏅 TOP 5 MODELS:")
for i in range(min(5, len(results_df_sorted))):
    row = results_df_sorted.iloc[i]
    rank = i + 1
    print(f"   {rank}. {row['model']} ({row['category']}) - F1: {row['f1']:.3f}, AUC: {row['auc']:.3f}")

print(f"\n💡 KEY INSIGHTS:")
print(f"   📊 Total Models Tested: {len(results_df_sorted)}")
print(f"   🚀 SOTA Models Tested: {len(results_df[results_df['category'] == '🚀 SOTA Enhanced'])}")
print(f"   🏆 Best F1 Score: {results_df_sorted.iloc[0]['f1']:.3f}")
print(f"   ⚡ Best AUC Score: {results_df_sorted.sort_values('auc', ascending=False).iloc[0]['auc']:.3f}")

# 다분야 기법 효과 분석
sota_models = results_df[results_df['category'] == '🚀 SOTA Enhanced']
if len(sota_models) > 0:
    print(f"\n🔬 SOTA TECHNIQUES ANALYSIS:")
    print(f"   📈 SOTA Average F1: {sota_models['f1'].mean():.3f}")
    print(f"   📈 SOTA Best F1: {sota_models['f1'].max():.3f}")
    print(f"   📈 SOTA Models in Top 5: {len([m for m in results_df_sorted.head(5)['category'] if '🚀' in m])}")
    
    # 각 SOTA 기법별 성능
    print(f"   🔥 Transformer-based: {sota_models[sota_models['model'].str.contains('Transformer')]['f1'].max():.3f}")
    print(f"   🎯 Contrastive Learning: {sota_models[sota_models['model'].str.contains('Contrastive')]['f1'].max():.3f}")
    print(f"   🧠 Meta-Learning: {sota_models[sota_models['model'].str.contains('Meta')]['f1'].max():.3f}")
    print(f"   🔄 Multi-Task: {sota_models[sota_models['model'].str.contains('MultiTask')]['f1'].max():.3f}")
    print(f"   🎨 Self-Supervised: {sota_models[sota_models['model'].str.contains('SelfSupervised')]['f1'].max():.3f}")

print(f"\n🎉 SOTA Enhancement Complete! Multiple cutting-edge techniques successfully integrated!")


🏆 COMPREHENSIVE COMPARISON RESULTS (SOTA Enhanced)


KeyError: 'traditional_ml'

In [35]:
import torch

# GPU 기본 정보
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU Count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"\n=== GPU {i} ===")
        print(f"Name: {torch.cuda.get_device_name(i)}")
        print(f"Total Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
        print(f"Allocated: {torch.cuda.memory_allocated(i) / 1e9:.2f} GB")
        print(f"Cached: {torch.cuda.memory_reserved(i) / 1e9:.2f} GB")
        print(f"Free: {(torch.cuda.get_device_properties(i).total_memory - torch.cuda.memory_reserved(i)) / 1e9:.2f} GB")

CUDA Available: True
GPU Count: 1

=== GPU 0 ===
Name: NVIDIA GeForce RTX 3070
Total Memory: 8.59 GB
Allocated: 0.14 GB
Cached: 22.22 GB
Free: -13.63 GB
